# Hate-speech Detection model (w/ Small, Fine-selected data)

In [14]:
!git clone https://github.com/joseph1723/CS376_Final_Project.git

fatal: destination path 'CS376_Final_Project' already exists and is not an empty directory.


In [15]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
import math
from keras.preprocessing.sequence import pad_sequences

In [16]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    item = self.df.iloc[idx, 1]
    return text, item

Hyperparameters

In [17]:
train_rate, test_rate = 0.9, 0.05
total_df = pd.read_csv('/content/CS376_Final_Project/augmented_data/Dataset_new_856.csv', sep=',')
total_df.dropna(inplace=True)
total_df = total_df[["text", "label"]]
total_df["label"] = [1 if i == "nothate" else 0 for i in total_df["label"]]

LSTM 데이타 전처리

In [18]:
total_dataset = TestDataset(total_df)
total_loader = DataLoader(total_dataset, batch_size=1, shuffle=True)

In [19]:
# clone the repo
!git clone https://github.com/richliao/textClassifier.git
# install Dependent library

fatal: destination path 'textClassifier' already exists and is not an empty directory.


In [20]:
# !cd textClassifier
# !ls
# !pip install -r req.txt
!pip install -r ./textClassifier/req.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached backports.weakref-1.0.post1-py2.py3-none-any.whl (5.2 kB)
  Using cached bleach-1.5.0-py2.py3-none-any.whl (17 kB)
  Using cached Cython-0.27.3.tar.gz (1.8 MB)
  Using cached enum34-1.1.6-py3-none-any.whl (12 kB)
  Using cached funcsigs-1.0.2-py2.py3-none-any.whl (17 kB)
  Using cached h5py-2.7.1.tar.gz (264 kB)
  Using cached html5lib-0.9999999.tar.gz (889 kB)
  Using cached Keras-2.0.8-py2.py3-none-any.whl (276 kB)
  Using cached laspy-1.5.0-py3-none-any.whl (489 kB)
  Using cached lda-1.0.5.tar.gz (303 kB)
  Using cached Markdown-2.6.9.tar.gz (271 kB)
  Using cached mock-2.0.0-py2.py3-none-any.whl (56 kB)
  Using cached nltk-3.3.0.zip (1.4 MB)
  Using cached numpy-1.13.3.zip (5.0 MB)
  Using cached olefile-0.44.zip (74 kB)
  Using cached pandas-0.20.3.tar.gz (10.4 MB)
  Using cached pbr-3.1.1-py2.py3-none-any.whl (99 kB)
  Using cached Pillow-4.3.0.tar.gz (13.9 MB)
  Usi

In [21]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-06-17 12:42:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-17 12:42:15--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-17 12:42:15--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [10]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [11]:
!wget https://www.kaggle.com/c/word2vec-nlp-tutorial/download/labeledTrainData.tsv

--2022-06-17 12:45:28--  https://www.kaggle.com/c/word2vec-nlp-tutorial/download/labeledTrainData.tsv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?returnUrl=%2Fcompetitions%2Fword2vec-nlp-tutorial [following]
--2022-06-17 12:45:28--  https://www.kaggle.com/account/login?returnUrl=%2Fcompetitions%2Fword2vec-nlp-tutorial
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘labeledTrainData.tsv’

labeledTrainData.ts     [ <=>                ]   6.56K  --.-KB/s    in 0.008s  

2022-06-17 12:45:28 (789 KB/s) - ‘labeledTrainData.tsv’ saved [6719]



In [12]:
!pip install --upgrade cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install --upgrade tensorflow==1.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==1.3.0 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.0+zzzcolab20220506153740, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.4+zzzcolab20220516125453, 2.6.5, 2.6.5+zzzcolab20220523104206, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.0+zzzcolab20220506150900, 2.7.1, 2.7.2, 2.7.2+zzzcolab20220516114640, 2.7.3, 2.7.3+zzzcolab20220523111007, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.0+zzzcolab20220506162203, 2.8.1, 2.8.1+zzzcolab20220516111314, 2.8.1+zzzcolab20220518083849, 2.8.2, 2.8.2+zzzcolab20220523105045, 2.8.2+zzzcolab20220527125636, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1)
ERROR: No

In [22]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np
import pandas as pd
import sys, os, importlib, re, tensorflow.python.keras.engine
from collections import defaultdict
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Embedding, Dropout, LSTM, Bidirectional, Embedding, Layer
from keras.models import Model
from keras import initializers

os.environ['KERAS_BACKEND']='theano'

#Parameters
MAX_SEQUENCE_LENGTH = 54
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


texts = []
labels = []
valid_lengths = []
split_input = []

for text, label in total_loader:
  texts.append(text[0])
  labels.append(label[0])
  valid_lengths.append(len(text[0].split(' ')))
  split_input.append(text[0].split(' '))
    

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index

textseq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
split = pad_sequences(split_input, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post', dtype = object, value = '_PAD_')

labels = to_categorical(np.asarray(labels))

indices = np.arange(textseq.shape[0])
np.random.shuffle(indices)


valid_lengths = np.array(valid_lengths)
split = np.array(split)
textseq = textseq[indices]
labels = labels[indices]
valid_lens = valid_lengths[indices]
split = split[indices]
nb_validation_samples = int(VALIDATION_SPLIT * textseq.shape[0])


texts_for_print = np.array(texts)
texts_for_print = texts_for_print[indices]


x_train = textseq[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = textseq[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]
vallen_train = valid_lens[:-nb_validation_samples]
vallen_val = valid_lens[-nb_validation_samples:]

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = self.init((input_shape[-1],))
        self.sm = tf.keras.layers.Softmax(axis = -1)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x, mask=None, splited_input = None, answer = None, batch_size = None):
        eij = K.tanh(K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1))
        masking = np.array([range(x.shape[1])])<mask
        ai = self.sm(eij, masking)
        ai_result = tf.argsort(ai, direction = 'DESCENDING',)
        weights = ai/tf.expand_dims(K.sum(ai, axis = 1), 1)
        weighted_input = x*tf.expand_dims(weights,2)
        return K.sum(weighted_input, axis = 1), ai_result[:, :3], ai


GLOVE_DIR = "./"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))



embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
#Input Declartion
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
valid_len_input = Input(shape = (1, ), dtype = 'int32')

#Model structure
embedded_sequences = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)(sequence_input)
lstm_out = Bidirectional(LSTM(100, return_sequences=True))(embedded_sequences)
att_out, _, _ = AttentionLayer()(lstm_out, mask = valid_len_input)
preds = Dense(2, activation='softmax')(att_out)
#Early stopper
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
#Model Train
model = Model([sequence_input, valid_len_input], preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
print("model fitting - attention Bi-LSTM network")
model.summary()
model.fit([x_train, vallen_train], y_train, validation_data=([x_val, vallen_val], y_val),
          batch_size = 50 , epochs = 500, callbacks=[callback])

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Total 400000 word vectors.
model fitting - attention Bi-LSTM network
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 54)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 54, 100)      376800      ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 54, 200)      160800      ['embedding[0][0]']              
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                         

In [25]:
model.save('./model_save_check_small')

INFO:tensorflow:Assets written to: ./model_save_check_small/assets


INFO:tensorflow:Assets written to: ./model_save_check_small/assets


In [26]:
!zip -r model_save_check_small.zip model_save_check_small

updating: model_save_check_small/ (stored 0%)
updating: model_save_check_small/variables/ (stored 0%)
updating: model_save_check_small/variables/variables.data-00000-of-00001 (deflated 13%)
updating: model_save_check_small/variables/variables.index (deflated 65%)
updating: model_save_check_small/keras_metadata.pb (deflated 89%)
updating: model_save_check_small/saved_model.pb (deflated 91%)
updating: model_save_check_small/assets/ (stored 0%)


In [ ]:
Test_uniform_df = pd.read_csv("/content/CS376_Final_Project/datasets/2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.2.csv",sep=',')
Test_uniform_df.dropna(inplace=True)
Test_uniform_df = Test_uniform_df[['text','label']]
Test_uniform_df_hate = Test_uniform_df.loc[Test_uniform_df['label'] == 'hate',:]
Test_uniform_df_nothate = Test_uniform_df.loc[Test_uniform_df['label'] == 'nothate',:]
Test_uniform_df = pd.concat([Test_uniform_df_hate[:200], Test_uniform_df_nothate[:200]], axis=0).reset_index(drop=True)
#Test_uniform_df = Test_uniform_df.sample(frac=1).reset_index(drop=True)
Test_uniform_df["label"] = [1 if i == "nothate" else 0 for i in Test_uniform_df["label"]]

print(Test_uniform_df)

texts_test = list(Test_uniform_df['text'])
results_True = list(Test_uniform_df['label'])

                                                  text  label
0    Any of those gas chambers still available? I w...      0
1    He will never propose to you, when you get old...      0
2    there is no difference between white women and...      0
3    The world would be so much better if we were d...      0
4    comparing maternity leave with paternity leave...      0
..                                                 ...    ...
395  I read an article where black men said it can ...      1
396  Honestly, I was pretty shocked to hear that sh...      1
397  I still believe all of that is because most of...      1
398  you can not come with us because you are weari...      1
399                               some men are abusers      1

[400 rows x 2 columns]


In [ ]:
valid_lengths_test = []
split_input_test = []
for text in texts_test:
  valid_lengths_test.append(len(text.split(' '))) 
  split_input_test.append(text.split(' '))
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index

data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
split_test = pad_sequences(split_input_test, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post', dtype = object, value = '_PAD_')

indicess = np.arange(data_test.shape[0])
# np.random.shuffle(indices)

valid_lengths_test = np.array(valid_lengths_test)
split_test = np.array(split_test)
textss = np.array(texts_test)

x_test = data_test[:]
vallen_test = valid_lengths_test[:]


In [ ]:
get_fil_tgt = K.function([model.layers[0].input, model.layers[3].input],
                                  [model.layers[4].output, model.layers[5].output])
x = [x_test, vallen_test]
[_, fil_tgt, ai], preds = get_fil_tgt(x)
result = preds.argmax(axis = -1)
print(result)

# for i in range(len(fil_tgt)):
#   # print(fil_tgt[0][1][i])
#   if result[i] == 0 and results_True[i] == 0:
#     print(split_test[i, fil_tgt[i]], textss[i], ai[i][fil_tgt[i]])
#   # else:
#   #   print(textss[i])

[1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1
 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 1 1 0
 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 1 0 1 0 0 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 0 1
 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1
 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 1
 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1]


In [ ]:
words = []
for i in range(len(fil_tgt)):
  # print(fil_tgt[0][1][i])
  if result[i] == 0 and results_True[i] == 0:
    words += list(split_test[i, fil_tgt[i]])
  
print(len(words))

from collections import Counter
occurence_count = Counter(words)
print(occurence_count.most_common(3))

120
[('fucking', 5), ('are', 5), ('rude', 4)]


# Scoring (400 samples w/ uniform-distribution)

In [ ]:
print(results_True)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 31.4 MB/s 


In [ ]:
from torchmetrics.functional import precision_recall
from torchmetrics import F1Score
from torchmetrics import Accuracy

#Scoring
#results_True / results
labels_true = torch.tensor(results_True)
labels_pred = torch.tensor(result)

acc = Accuracy()
accuracy = acc(labels_pred, labels_true)

precision, recall = precision_recall(labels_pred, labels_true)
f1 = F1Score()
f1_score = f1(labels_pred, labels_true)

print(accuracy)
print(precision, recall)
print(f1_score)

tensor(0.3775)
tensor(0.3775) tensor(0.3775)
tensor(0.3775)


In [ ]:
crr = list(labels_pred==labels_true)
print(([i for i in range(len(crr)) if crr[i]]))

[6, 9, 16, 23, 31, 34, 35, 38, 51, 53, 56, 71, 72, 74, 84, 87, 92, 106, 119, 121, 124, 131, 136, 142, 143, 150, 153, 155, 158, 161, 163, 164, 167, 170, 174, 179, 181, 183, 187, 195, 200, 201, 208, 210, 215, 216, 218, 219, 220, 223, 224, 225, 235, 236, 237, 239, 240, 242, 244, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 260, 262, 265, 269, 270, 271, 275, 276, 277, 278, 280, 281, 283, 284, 285, 287, 288, 291, 292, 295, 296, 297, 300, 303, 304, 305, 306, 307, 308, 309, 311, 312, 317, 319, 320, 321, 322, 323, 325, 327, 329, 330, 332, 333, 334, 336, 337, 338, 339, 342, 344, 346, 347, 351, 352, 353, 354, 356, 357, 358, 359, 363, 364, 366, 369, 372, 373, 376, 377, 382, 385, 386, 389, 390, 393, 394, 395, 396, 397, 398, 399]


#Scoring Done.